<a href="https://colab.research.google.com/github/jaalvalcan/GEE_index_sets/blob/main/TEST_AirTemp2mt_band_nir(8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import ee

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-jaalvalcan')

In [ ]:
import geemap
import geopandas as gpd

In [ ]:
AOI = "/content/sample_data/shitoukoumen.shp"

gdf = gpd.read_file(AOI)
print(gdf.head())
print(gdf.crs)
print(gdf.total_bounds)


In [ ]:
# Convert the GeoDataFrame to an Earth Engine FeatureCollection
ee_fc = geemap.geopandas_to_ee(gdf)

# Create a Map
Map = geemap.Map(center=[0, 0], zoom=2)

# Add the AOI layer
Map.addLayer(ee_fc, {'color': 'red'}, 'AOI')

# Center the map on the AOI
Map.centerObject(ee_fc, 10)

# Display the map
Map


In [ ]:


# Sentinel-2 SR Harmonized collection ID
S2_COLLECTION = 'COPERNICUS/S2_SR_HARMONIZED'  # [COPERNICUS/S2_SR_HARMONIZED]

# Dates: 1-2-2024 to 1-12-2024 (YYYY-MM-DD)
start_date = '2023-10-01'
end_date = '2023-10-30'

# Filter Sentinel-2 SR Harmonized by date, AOI and cloudiness <= 20%
collection = (
    ee.ImageCollection(S2_COLLECTION)
    .filterDate(start_date, end_date)
    .filterBounds(ee_fc)
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20))
)

# Create median composite and clip to AOI
s2_median = collection.median().clip(ee_fc)

# Visualization parameters (true color)
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.2,
}

# If you already have Map created earlier, reuse it.
# Otherwise, create a new Map:
# Map = geemap.Map(center=[0, 0], zoom=2)

Map.addLayer(s2_median, vis_params, 'S2 Median 2024 (<=20% clouds)')
Map.addLayer(ee_fc, {'color': 'red'}, 'AOI')
Map.centerObject(ee_fc, 10)

Map


In [ ]:
S2_COLLECTION = 'COPERNICUS/S2_SR_HARMONIZED'
year = 2023

months = list(range(1, 13))
stats = []

for m in months:
    start = ee.Date.fromYMD(year, m, 1)
    end = start.advance(1, 'month')

    ic = (ee.ImageCollection(S2_COLLECTION)
          .filterDate(start, end)
          .filterBounds(ee_fc))

    mean_cloud = ic.aggregate_mean('CLOUDY_PIXEL_PERCENTAGE')
    stats.append({
        'month': m,
        'mean_cloudiness': ee.Number(mean_cloud).getInfo()
    })

stats


In [ ]:
for s in stats:
    print(f"Month {s['month']:02d}: {s['mean_cloudiness']:.2f}% clouds")

# Optionally: sort by cloudiness (ascending)
stats_sorted = sorted(stats, key=lambda x: x['mean_cloudiness'])
print("\nMonths from clearest to cloudiest:")
for s in stats_sorted:
    print(f"Month {s['month']:02d}: {s['mean_cloudiness']:.2f}%")


In [ ]:
# Select the NIR band (B8) from the Sentinel-2 SR Harmonized median composite
nir_band = s2_median.select('B8')

# Visualization parameters for NIR (stretch values as needed)
nir_vis = {
    'bands': ['B8'],
    'min': 0,
    'max': 3000,   # adjust if image looks too dark/bright
    'palette': ['0000ff', 'ffffff']  # dark blue (low reflectance) to white (high)
}

# Add NIR band layer to the map
Map.addLayer(nir_band, nir_vis, 'NIR (B8) - Water contrast')

# Keep AOI on top for reference
Map.addLayer(ee_fc, {'color': 'red'}, 'AOI')

Map.centerObject(ee_fc, 10)
Map


In [ ]:
import os

In [ ]:
# 2. Load the Local Shapefile as an ee.FeatureCollection
# Shitoukoumen AOI path
# The ee_fc variable was already created from the shapefile earlier in the notebook. Reusing it.
# shp_path = "/content/sample_data/shitoukoumen.shp"
# aoi = geemap.shp_to_ee(shp_path)

# 3. Load ERA5-Land Daily Aggregates for October 2023
era5_land = (
    ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    .select('temperature_2m')
    .filterDate('2023-10-01', '2023-10-31')
    .filterBounds(ee_fc)
)

# 4. Calculate Monthly Mean and CLIP to AOI
# Subtract 273.15 to convert from Kelvin to Celsius
# Check if the image collection is empty before proceeding
if era5_land.size().getInfo() == 0:
    print("No ERA5-Land images found for the specified date range and AOI. Skipping temperature layer.")
else:
    monthly_mean_celsius = era5_land.mean().subtract(273.15).clip(ee_fc)

    # 5. Define Visualization Parameters (Updated for Celsius range)
    vis_params = {
        'min': -10,
        'max': 35,
        'palette': [
            '000080', '0000d9', '4000ff', '8000ff', '0080ff', '00ffff', '00ff80',
            '80ff00', 'daff00', 'ffff00', 'fff500', 'ffda00', 'ffb000', 'ffa400',
            'ff4f00', 'ff2500', 'ff0a00', 'ff00ff'
        ]
    }

    # 6. Create Interactive Map and Display
    # Map is already initialized from previous cells. Reusing it.
    # Map = geemap.Map()
    Map.centerObject(ee_fc, 10)
    Map.addLayer(monthly_mean_celsius, vis_params, 'Monthly Mean Temp (°C) Oct 2023')
    Map.addLayer(ee_fc, {'color': 'red'}, 'AOI Boundary', False)

Map